# Cómo utilizar las políticas de confianza con los roles de IAM

Existen varios escenarios diferentes en los que se pueden utilizar los roles de IAM en AWS:

* **Un servicio o recurso de AWS accede a otro recurso de AWS en su cuenta** - Cuando un recurso de AWS necesita acceder a otros servicios, funciones o recursos de AWS, puede crear un rol que tenga los permisos adecuados para que lo utilice ese recurso de AWS. Servicios como AWS Lambda y Amazon Elastic Container Service (Amazon ECS) asumen roles para entregar credenciales temporales a su código que se ejecuta en ellos.

* **Un servicio de AWS genera credenciales de AWS para ser utilizadas por dispositivos que se ejecutan fuera de AWS** -
Las instancias híbridas de AWS IAM Roles Anywhere, AWS IoT Core y AWS Systems Manager pueden entregar credenciales de sesión de rol a aplicaciones, dispositivos y servidores que no se ejecutan en AWS.

* **Una cuenta de AWS accede a otra cuenta de AWS** - Este caso de uso se conoce comúnmente como un patrón de rol entre cuentas. Permite que los responsables de IAM humanos o mecánicos de una cuenta de AWS asuman este rol y actúen sobre los recursos de una segunda cuenta de AWS. Se asume un rol para permitir este comportamiento cuando el recurso en la cuenta de destino no tiene una política basada en recursos que podría utilizarse para conceder acceso entre cuentas.

* **Un usuario final autenticado con un proveedor de identidad web u OpenID Connect (OIDC) necesita acceder a sus recursos de AWS** - Este caso de uso permite que las identidades de Facebook o de proveedores de OIDC como GitHub, Amazon Cognito u otros proveedores genéricos de OIDC asuman un rol para acceder a los recursos de su cuenta de AWS.

* **Un cliente realiza la autenticación de la fuerza de trabajo mediante la federación de SAML 2.0** - Esto ocurre cuando los clientes federan a sus usuarios en AWS desde su proveedor de identidades corporativas (IdP), como Okta, Microsoft Azure Active Directory o Active Directory Federation Services (ADFS), o desde el Centro de identidades de AWS IAM (sucesor de AWS Single Sign-On).

In [1]:
import boto3
import json

In [2]:
# Create IAM client
iam = boto3.client('iam')

In [17]:
policy_ro = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "ec2:Describe*",
                "rds:Describe*",
                "sts:AssumeRole"
            ],
            "Resource": "*"
        }
    ]
}

In [19]:
# Create a policy
response = iam.create_policy(
    PolicyName='ReadOnlyAccessPolicy',
    PolicyDocument=json.dumps(policy_ro)
)

In [8]:
arn_policy = response['Policy']['Arn']
print("ARN Policy:", arn_policy)

ARN Policy: arn:aws:iam::925404979403:policy/ReadOnlyAccess


In [21]:
account_id = boto3.client('sts').get_caller_identity().get('Account')

In [24]:
role_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::" + str(account_id) + ":root"
            },
            "Action": "sts:AssumeRole",
            "Condition": {}
        }
    ]
}

In [27]:
# Create a role
response = iam.create_role(
    RoleName='ReadOnlyAccessRole',
    AssumeRolePolicyDocument=json.dumps(role_policy)
)

In [29]:
role_arn = response['Role']['Arn']
print("Role ARN:", role_arn)

Role ARN: arn:aws:iam::925404979403:role/ReadOnlyAccessRole


In [30]:
# Attach policy to role
response = iam.attach_role_policy(
    RoleName='ReadOnlyAccessRole',
    PolicyArn=arn_policy
)

In [31]:
# Switch to the role
sts_client = boto3.client('sts')
assumed_role_object = sts_client.assume_role(
    RoleArn=role_arn,
    RoleSessionName="AssumeRoleSession1"
)

In [40]:
access_key = assumed_role_object['Credentials']['AccessKeyId']
secret_key = assumed_role_object['Credentials']['SecretAccessKey']
session_token = assumed_role_object['Credentials']['SessionToken']

In [41]:
! aws configure set aws_access_key_id $access_key
! aws configure set aws_secret_access_key $secret_key
! aws configure set aws_session_token $session_token